# Macros: scans, data acquisition and storage
<br>
<br>
<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />
Sardana-Training by ALBA Synchrotron is licensed under the Creative Commons Attribution 4.0 International License.  
To view a copy of this license, visit http://creativecommons.org/licenses/by/4.0/.

## Contents

* MacroServer overview
* MeasurementGroup overview
* Generic Scan Framework
* Recorders

## [MacroServer overview](http://www.sardana-controls.org/devel/overview/overview_macroserver.html)

* Is a controlled environment to run procedures, called macros, sequentially or simultaneously (using multiple entry-points called *Doors*)
* Provides a standard [catalogue](http://www.sardana-controls.org/users/standard_macro_catalog.html) of procedures.
* Macros syntax is inspired on the SPEC control system syntax
* Allows to plug in new/custom procedures written as Python functions or classes
* Connects to device Pool(s) and allows to act on their elements from within macros

## [MeasurementGroup overview](http://www.sardana-controls.org/devel/overview/overview_measurementgroup.html)

* MeasurementGroup is a high level element whose role is to:
  * coordinate a complex measurement process
  * provide a simplified interface to it
* Run `uct` macro in Spock
  ```
  uct
  ``` 
* Open `expconf` in Spock
  ```
  expconf
  ```
* Disable some channels in the `expconf` e.g. `ct02` and `ct04` (Enabled column -> False) and run `ct` macro in spock:
  ```
  ct
  ```
* Briefly review other configuration parameters
* Add 1D channel `oned01` in `expconf` (Add channel `+` button), apply changes and run `ct` macro
* Create new measurement group using the `defmeas` macro in Spock and run `ct` afterwards:
  ```
  defmeas mntgrp02 ct01 twod01
  ct
  ``` 
  * Note the _external changes_ pop-up.
* Change active measurement group in expconf and run `ct` macro

## Generic Scan Framework - demo

* Precise software synchronized continuous scans require intensive monitoring loops. Increase the frequency of position checks (in Spock) and restart the Sardana server:
```
Pool_demo1_1.put_property({"MotionLoop_StatesPerPosition":"1"})
```
* Start taurustrend with high frequency of polling (in bash):
 ```
 taurus tpg trend mot01/position "eval:int(bool({ct01/state}))"
 ```
 * Configure *Forced read* period of 10 ms 
 * Configure fill area of the state curve and move it to y2 axis in *Plot configuration*
* Demonstrate step scan in Spock:
  ```
  ascan mot01 0 10 10 1
  ```
* Demonstrate continuous scan with no latency time in Spock:
  ```
  ascanct mot01 0 10 10 1
  ```
* Demonstrate continuous scan with 100 ms latency time (in spock):
  ```
  ascanct mot01 0 10 10 1 0.1
  ```
* Configure hardware synchronization for `ct01` in `expconf` (_Synchronizer_ column -> `tg01`) and execute the scan without latency
time:
  ```
  ascanct mot01 0 10 10 1
  ```
  * Note about cooexistence of software and hardware synchronization modes in the same scan
  * Note the continuous _moving_ state of the `ct01`

## Generic Scan Framework (GSF)

* Scanning modes: step, continuous and time
* Provides the catalogue of turn-key scan macros e.g., `ascan`, `dscan`, `a2scanc`, `ascanct`, `mesh`, `timescan`, etc.
* Provides the base classes for developing custom scans e.g., `GScan`, `SScan`, `CScan`, etc.
* Custom scans are developed by instantiating the base class and developing a custom generator
* Scans are highly configurable with the [environment variables](http://www.sardana-controls.org/users/scan.html#configuration) e.g. `ActiveMntGrp`, `ScanFile`, `ApplyInterpolation`
* Scans macros [support hooks](http://www.sardana-controls.org/devel/howto_macros/scan_framework.html#hooks-support-in-scans) e.g. `pre-scan`, `post-acq`

## [Recorders](http://www.sardana-controls.org/devel/howto_recorders.html#writing-recorders)

![Sardana data flow](res/recorders.png)

## Recorders - demo

### Demonstrate recorder selection based on the file extension

* In spock start `expconf`
* In `expconf` _Storage_ tab set scan file to `test.h5` and directory to `/tmp` (equivalent in Spock with `senv ScanFile test.h5` and `senv ScanDir /tmp`)
* Execute a scan: `ascan mot01 0 2 2 0.1`
* Access to the data with `silx`
* Close the `silx`